In [47]:
import os
import subprocess
import sys

### Notebook for generating Transcript count from paired end raw FASTQ files

__Required program for this notebook__
* STAR aligner https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3530905/
  * STAR aligner was chosen due to ease of use and benchmarking done by  
    https://www.ecseq.com/support/ngs/best-RNA-seq-aligner-comparison-of-mapping-tools
  * Pre-built executables can be downloaded from here https://github.com/alexdobin/STAR/tree/master/bin  
  
__Required files for this notebook__
* Reference FASTA file
  * S288C genome latest release from https://downloads.yeastgenome.org/sequence/S288C_reference/genome_releases/
    * Chromosome names needed to renamed to match with Annotation downloaded below.
    * Use the provided code below in cell to rename the chromosomes
  * Saccharomyces cerevisiae annotation file from ftp://ftp.ensembl.org/pub/release-99/gtf/saccharomyces_cerevisiae/
  
__Required scripts for this notebook__
* run_generate_star_index.sh
  * Script will generate STAR genome index. Genome index is required for running alignment
* run_star_aligner.sh
  * Script takes STAR genome index generated above and paired end FASTQ files(R1, R2) and generates  
    Transcript alignment sorted BAM file.
    

In [92]:
workdir = './'

### If you need to rename the Reference fasta, uncomment the command below and run the cell

__Be sure to change the input and output FASTA files accordingly based on your working directory!!!__

In [93]:
# Rename the FASTA file to match with annotation
# >ref|NC_001133| [org=Saccharomyces cerevisiae] [strain=S288C] [moltype=genomic] [chromosome=I]

def rename_chromosomes(input_fasta, output_fasta):
    with open(input_fasta) as infasta, open(output_fasta, 'w') as outfasta:
        for line in infasta:
            if 'mitochondrion' in line:
                new_reference_name = '>chrMito'
                outfasta.write(new_reference_name + '\n')
            elif line.startswith('>'):
                new_reference_name = '>chr' + line.split('chromosome=')[-1][:-2]
                outfasta.write(new_reference_name + '\n')
            else:
                outfasta.write(line)

                
input_fasta = os.path.join(workdir, 'S288C_reference/S288C_reference_sequence_R64-2-1_20150113.fsa')
output_fasta = os.path.join(workdir, 'S288C_reference/S288C_reference_sequence_chr_renamed.fsa')
rename_chromosomes(input_fasta, output_fasta)                

### Generating the STAR aligner index

In [94]:
STAR_EXECUTABLE='/Users/jganbat/program_sources/STAR/bin/MacOSX_x86_64/STAR'

In [96]:
run_generate_star_index = 'run_generate_star_index.sh'

NUM_THREAD=4
REF_FASTA=os.path.join(workdir, 'S288C_reference/S288C_reference_sequence_chr_renamed.fsa')
INDEX_DIR='S288_STAR_index'
ANNOTATIONS=os.path.join(workdir, 'Saccharomyces_cerevisiae.R64-1-1.99.gtf')
READLEN=150

command = ['bash', run_generate_star_index, STAR_EXECUTABLE, REF_FASTA, ANNOTATIONS, INDEX_DIR, str(READLEN)]

if subprocess.call(command) != 0:
    raise Exception('Index generation failed check Log.out') 

### Run STAR alignment

In [97]:
FASTQ_R1 = 'test_fastqs/test_r1.fastq.gz'
FASTQ_R2 = 'test_fastqs/test_r2.fastq.gz'
OUTDIR = 'test_alignment/'
run_star_aligner = 'run_star_aligner.sh'
command = ['bash', run_star_aligner, STAR_EXECUTABLE, INDEX_DIR, FASTQ_R1, FASTQ_R2, OUTDIR]

if subprocess.call(command) != 0:
    raise Exception('Alignment failed, check Log.out') 

### Count Transcript and write to file

In [98]:
import pysam 
from collections import defaultdict

min_mapq = 255

transcript_bam_file = os.path.join(OUTDIR, 'Aligned.toTranscriptome.out.sorted.bam')
transcript_counts_dict = defaultdict(int)
with pysam.AlignmentFile(transcript_bam_file) as tx_bam:
    for record in tx_bam.fetch():
        if record.mapping_quality >= min_mapq:
            transcript_counts_dict[record.reference_name] += 1
            
out_transcript_count = 'test_transcript_count/test_counts.tsv'

with open(out_transcript_count, 'w') as outfile:
    outfile.write('gene\tcount\n')
    for transcript_name in sorted(transcript_counts_dict.keys()):
        transcript_count = transcript_counts_dict[transcript_name]
        outfile.write('{}\t{}\n'.format(transcript_name, transcript_count))

### Example transcript counts

In [99]:
import pandas as pd

transcript_counts_df = pd.read_csv(out_transcript_count, sep='\t')
transcript_counts_df.head()

,gene,count
0,LSR1_snRNA,144
1,NME1_snoRNA,4
2,RPR1_ncRNA,6
3,SCR1_ncRNA,280
4,YAL003W_mRNA,6
